## Imports for the program (Original program by Josh Lozjim and edited by Hayden Fuss, this version by Jeremy McKenzie)

In [46]:
%matplotlib inline
import matplotlib
import time
import csv
import numpy as np
import matplotlib.pyplot as plt
import math
import sys

time_fmt declares the format for the time data
year_data declares an empty hash, and the for loop creates further hashes (think: vectors) assigned to the keys of '2012' and '2013'

In [ ]:
time_fmt = "%m/%d/%Y %H:%M"

def getTimeString(currentHour):
    timeStr = ""
    if currentHour != 12 and currentHour != 24:
        timeStr += str(currentHour%12)
    else:
        timeStr += str(12)
    timeStr += ":00"
    if currentHour in range (0,12) or currentHour == 24:
        timeStr += ' AM'
    else:
        timeStr += ' PM'
    return timeStr

years = ('2012', '2013')

year_data = {}
filename = raw_input("Enter the file name to read data from: ")
#filename = sys.stdin.readline()
print filename
outputfile = raw_input("Enter the file name to write data to: ")
#outputfile = sys.stdin.readline()
print outputfile
filename = filename.rstrip("\n")
outputfile = outputfile.rstrip("\n")

Enter the file name to read data from: 911_medical_2012_2013.csv
911_medical_2012_2013.csv


This portion of the program builds the data list, organized by month, in a list called emergency. It runs through the list, pulling dates from the csv file, and then counts the number of times any particular month (and thereby the amount of calls in that month) comes up. If a month has not previously been encountered, it is added to the list and given a count of one. Otherwise, it's incremented by one.

In [ ]:
# for y in years:
    # year_data[y] = {}   
with open(filename) as csvfile:
    threeOne = csv.DictReader(csvfile)
        # for all the 911 calls the reader finds
    for e in threeOne:
        date = time.strptime(e['ENTRY_DT'], time_fmt) # converts string version of date into a date object
        y = date.tm_year
        hour = date.tm_hour
        if not y in year_data.keys():
            year_data[y] = {}        
        if date.tm_mon == 4:
            if (y == 2012 and date.tm_mday == 16) or (y == 2013 and date.tm_mday == 15): 
                # looking for 4/16/12 & 4/15/13 -- marathon monday 2012 & 2013
                if hour not in year_data[y].keys():
                    year_data[y][hour] = 1
                else:
                    year_data[y][hour] += 1

for h in year_data[2012].keys():
    if not h in year_data[2013].keys():
        year_data[2013][h] = 0
        
for h in year_data[2013].keys():
    if not h in year_data[2012].keys():
        year_data[2012][h] = 0
        
        
for y in (2012, 2013):
    for h in range(0,24):
        if not h in year_data[y].keys():
            year_data[y][h] = 0
    for h in sorted(year_data[y]):
        if h != 0:
            year_data[y][h] += year_data[y][h-1]

This section adjusts the data to plot-friendly tuples (again, like vectors) while the last line ensures that the bar will not be overrun by the data. It does this by finding the largest humber in either list, divides that by decimal 1000, rounds that up and converts to an integer, and mutiplies that by 1000.

In [ ]:
ind=np.arange(len(year_data[2012]))
width=.35

tick_labels = []

counts_2012 = []

for apr_h in sorted(year_data[2012]):
    counts_2012.append(year_data[2012][apr_h])
    tick_labels.append(getTimeString(apr_h+1))
    
counts_2013 = []

for apr_day in sorted(year_data[2013]):
    counts_2013.append(year_data[2013][apr_day])

maxCount = int(math.ceil(max(counts_2012 + counts_2013) / 100.0)) * 100

This is the section that organizes the bar graph itself.
The last function puts a label with the exact number on the top of the bar graph

In [ ]:

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)

rects1 = ax.bar(ind, counts_2012, width, color='r')

rects2 = ax.bar(width+ind, counts_2013, width, color='y')

ax.set_xlim(-width, len(ind) + width)
ax.set_ylim(0, maxCount)
ax.set_ylabel('Number of 911 Calls')
ax.set_title('911 Incremental Call Volume by Hour on Marathon Monday')
ax.set_xticks(ind+width)

tickNames = ax.set_xticklabels(tick_labels)
plt.setp(tickNames, rotation=45, fontsize=10)

ax.legend((rects1[0], rects2[0]), years)

def autolabel(rects):
# attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x()+rect.get_width()/2., 1.05*height, '%d'%int(height),
            ha='center', va='bottom')

#autolabel(rects1)
#autolabel(rects2)

plt.savefig(outputfile, dpi=96)
plt.show()